In [1]:
import fitz
import pytesseract
from PIL import Image
import io
import camelot
import pandas as pd

In [29]:
def extract_pdf_content(pdf_file):
    """
    Will be used to extract text, images and tables from a pdf document
    Will also be used to extract text, images and tables from a scanned PDF using OCR
    """

    pdf_content = {
        "text": [],
        "images": [],
        "tables": []
    }

    document = fitz.open(pdf_file)

    for page_num, page in enumerate(document):
        page_text = page.get_text() # extracting text from the pdf
        if page_text.strip():  # If text exists, PDF is processed as text-based
            try: # extracting tables as pdfs and appending to the list in dictionary
                tables = camelot.read_pdf(pdf_file,pages=str(page_num+1),flavor="lattice")
                for table in tables:
                    df = pd.DataFrame(table.df.values)
                    df.columns = df.iloc[0]  # setting the first row as column headers
                    df = df.drop(0).reset_index(drop=True)
                    pdf_content["tables"].append(df)
                    # Extract the raw table text and remove it from the page text
                    # table_text = "\n".join(["\t".join(map(str, row)) for row in table.df.values])
                    # page_text = page_text.replace(table_text, "")
            except Exception as e:
                print(f"No tables found on page {page_num+1}: {e}")
            pdf_content["text"].append(page_text)

            for img_index, img in enumerate(page.get_images(full=True)): # extracting images from 
                xref = img[0]
                base_image = document.extract_image(xref)
                img_bytes = base_image["image"]
                img_extension = base_image["ext"]  # to get image format to preserve transparency if needed
                # Load the image with Pillow
                image = Image.open(io.BytesIO(img_bytes))
                # Preserve transparency if the image has an alpha channel
                if image.mode in ("RGBA", "LA"):
                    image = image.convert("RGBA")
                else:
                    image = image.convert("RGB")
                # Save the image in its original format
                img_filename = f"{pdf_file}_page{page_num+1}_img{img_index+1}.{img_extension}"
                image.save(img_filename, format=img_extension.upper())
                pdf_content["images"].append(img_filename)
        else:
            # If no text, process as scanned PDF using OCR
            pix = page.get_pixmap()  # Render page to an image
            img = Image.open(io.BytesIO(pix.tobytes()))
            ocr_text = pytesseract.image_to_string(img)
            pdf_content["text"].append(ocr_text)
    document.close()
    return pdf_content

In [35]:
# Example usage
pdf_file = "/Users/kashishmandhane/Documents/Kashish Data/LAPTOP STUFF/DJ Sanghvi College/Extra-curriculars/Hackathons/Ed-tech/test_file.pdf"

In [36]:
extract_pdf_content(pdf_file)

{'text': ['My name is Kashish, YAY!!! \nLol, this is so fun, hahahaha! \n \n \n \n \nName \nis \nKashish \nA  \ntest \ntable \ntext \nAnd  \ntable \nand \nPyMuPDF \nlol \n \n \nHi \nBye \nMy \nName \nis \nKashish \nMandhane \nthis \nis \nA  \ntest \ntable \nto \ncheck \nfor \ntext \nAnd  \ntable \nextraction \nUsing  \ncamelot \nand \nPyMuPDF \nlol \n \n \nHi \nBye \nMy \nMandhane \nthis \nis \nto \ncheck \nfor \nextraction \nUsing  \ncamelot \n \n'],
 'images': ['/Users/kashishmandhane/Documents/Kashish Data/LAPTOP STUFF/DJ Sanghvi College/Extra-curriculars/Hackathons/Ed-tech/test_file.pdf_page1_img1.png',
  '/Users/kashishmandhane/Documents/Kashish Data/LAPTOP STUFF/DJ Sanghvi College/Extra-curriculars/Hackathons/Ed-tech/test_file.pdf_page1_img2.png'],
 'tables': [0  Name       is Kashish
  0     A     test   table
  1  text      And   table
  2   and  PyMuPDF     lol,
  0          Hi    Bye       My  Name       is Kashish
  0    Mandhane   this       is     A     test   table
  1   